## Описаение 
Данный блок производт парсинг обучающего набора данных для проекта [Проект 5. Выбираем авто выгодно](https://lms.skillfactory.ru/courses/course-v1:Skillfactory+DST-12+11MAR2020/courseware/c16441cf4f0a4f8486955f2be47f1cf0/67e09c15d9cd4b8eb691417898f4dfc2/1?activate_block_id=block-v1%3ASkillfactory%2BDST-12%2B11MAR2020%2Btype%40vertical%2Bblock%40f820a06d37d84dd98ab6acaa03392b5e)

Данные сбираются на сайте https://auto.ru/,  
из них формируется *pd.DataFrame* со следуюшими колонками:   
  
 **bodytype** -   наименование типа кузова  
**brand** -  наименование марки  
**car_url** -  ссылка на обьявления о продаже  
**color** -  ссылки на обьявления о продаже  
**engineDisplacement** -  обьем двигателя  
**enginePower** -  мощность двигателя  
**equipment_dict** -  словарь с перечислением оснащения автомобиля.  
**fuel_type** -  тип топлива  
**mileage** -  пробег авто  
**modelDate** -  год начала выпуска модели  
**model_name** -  наименование модели  
**numberOfDoors** -  количество дверей  
**productionDate** -  Год производства автомобиля  
**sell_id** -  содержит id обьявления   
**vehicleTransmission** -  содержит id обьявления  
**vendor** -  обобщающий признак: принадлежность марки к европейским либо японским маркам  
**Владельцы** -  количество владельцев авто  
**ПТС** -  Колонка содержит ('Оригинал', 'Дубликат') категорию ПТС  
**Привод** -   категория привода  
**Руль** -   категрию право- или левосторонности управления  
**offerprice** - цена продпажи (целевая переменная)

# Импорт библиотек, установка параметров, определение функций
## Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import requests
import time
import re

from tqdm import tqdm
from bs4 import BeautifulSoup

## Функции

In [2]:
def get_marks_models():
    '''
    возврашщает pd.Series в котором 
    индекс - название марки автомобиоя, 
    значения - списки названий моделей для каждой маркию
    
    '''
    marks_models = pd.Series() 

    for pages_num in range(1,20):
        if pages_num==1: params = {'view_type': 'list'}
        else:            params = {'page_num': pages_num,'view_type': 'list'}
    
        res = requests.get(url_for_marks_models, params=params, headers=headers)
        time.sleep(0.5)
        
        # создаем обьект bs4.BeautifulSoup
        soup = BeautifulSoup(res.text, 'html.parser')
        
        # список html-ек марок на странице
        marks_on_page_list = soup.find_all('dd', class_='catalog-all-text-list__desc') 
    
        if not marks_on_page_list: 
            break
        
        for mark in marks_on_page_list:
            models_of_mark = mark.find_all('a', class_='link_theme_auto') # список html-ек моделей марки
        
            link_for_mark_name = models_of_mark[0].get('href')  # линк первой модели, содержит обозначене марки на auto.ru
            mark_start = link_for_mark_name.find('cars/') + 5   # первый символ обозначеня марки
            mark_end = link_for_mark_name.find('/', mark_start) # последний символ обозначения марки
            mark_name = link_for_mark_name[mark_start:mark_end].upper() # получение обозначения марки      
        
            models_list = []
            for model in models_of_mark:
                link_theme_auto = model.get('href')                 # линк модели, содержит обозначение модели на auto.ru
                model_start = mark_end+1                            # первый символ обозначения модели
                model_end = link_theme_auto.find('/', model_start)  # последний символ обозначения модели
                model_name = link_theme_auto[model_start:model_end].upper() # получение обозначения модели
                models_list.append(model_name)
      
            marks_models[mark_name] = models_list
    return marks_models

def get_generation_year(model_url):
    '''
    возврашщает pd.DataFrame  в котором: 
        full_name - полное название модели с указанием поколения, 
        bodytype - тип кузова поколения модели
        generation_year - год начала выпуска поколения
        
    model_url - ссылка на страницу со списокм поколений модели
    
    '''        
    # получем html 
    models_list_res = requests.get(model_url+'?output_type=models_list')
    models_list_res.encoding = 'utf-8'
    
    # создаем обьект bs4.BeautifulSoup из html 
    models_list_bs = BeautifulSoup(models_list_res.text, 'html.parser') 
    
    # получаем список html-ек описаний для всех поколений модели  
    posting_tag = models_list_bs.find_all('div',class_ = "ListingItemGroup")
    
    # формируем списки с указанием поколения модели, типом кузова поколения и годом начала выпуска поколения
    # (необходимость указания типа кузова вызвана тем, что разные кузова зачастую переходят на следующее 
    #  поколение в разные годы)
    data = [
    [tag.find('a',class_ = 'ListingItemTitle-module__link').text,             # поколение
     tag.find('div',class_ = "ListingItemTitle-module__subtitle").text[:      # кузов
                tag.find('div',class_ = "ListingItemTitle-module__subtitle").text.find(' •')],
     int(tag.find('div',class_ = "ListingItemTitle-module__subtitle").text    # год
                [tag.find('div',class_ = "ListingItemTitle-module__subtitle").text.find('(')+1:
                 tag.find('div',class_ = "ListingItemTitle-module__subtitle").text.find('(')+5])] 
        for tag in posting_tag]
    
    result = pd.DataFrame(data,columns = ['full_name','bodytype','generation_year'])

    return result

def get_model_generation_year(marks_models_for_parsing):
    '''
    возврашщает pd.DataFrame  в котором: 
        full_name - полное название марки и модели с указанием поколения 
        bodytype - тип кузова поколения модели
        generation_year - год начала выпуска поколения
    
    marks_models_for_parsing - pd.Series в котором
        индекс: марка автомобиля
        значение: список моделей данной маркеи
    '''
    model_generation_year = pd.DataFrame(columns = ['full_name','bodytype','generation_year'])
    
    for mark in marks_models_for_parsing.index:
        print(mark, end=' | ')
        for model in marks_models_for_parsing[mark]:
            model_url = 'https://auto.ru/moskva/cars/' + mark.lower() + '/' + model.lower() + '/used/'
            tmp = get_generation_year(model_url)
            model_generation_year = model_generation_year.append(tmp)
            time.sleep(0.2)
    return model_generation_year

def get_features_from_ticket(ticket_url):
    '''
    возвращает pd.Series с признакамии полученными из карточки обьявления
    
    ticket_url: str, ссылка на страницу обьявления
        
    '''
    features = pd.Series(index = data_columns)
    
    # получем html карточки текщего обьявления
    ticket_res = requests.get(ticket_url)
    ticket_res.encoding = 'utf-8'
    
    # создаем обьект bs4.BeautifulSoup из html карточки текщего обьявления
    ticket_bs = BeautifulSoup(ticket_res.text, 'html.parser')  
    # проверка корректности резкльтата BeautifulSoup
    if ticket_bs:
    
    # получение признаков из карточки текущего обьявления
    #
        # bodyType      
        try: features['bodyType'] = ticket_bs.find('li',class_='CardInfoRow_bodytype').find('a').text
        except Exception: features['bodytype'] = np.NaN
           
        # brand        
        try: features['brand'] = ticket_bs.find_all('a',class_='CardBreadcrumbs__itemText')[0].text.strip().upper()
        except Exception: features['brand'] = np.NaN
    
        # car_url
        features['car_url'] = ticket_url
    
        # color        
        try: features['color'] = ticket_bs.find('li',class_='CardInfoRow_color').find('a').text
        except Exception: features['color'] = np.NaN
    
        #engineDisplacement            
        try: 
            engineDisplacement = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[0]
            features['engineDisplacement'] =  re.sub("[^\d.]", "", engineDisplacement)
        except Exception: features['engineDisplacement'] = np.NaN
          
        # enginePower
        try: 
            enginePower = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[1]
            features['enginePower'] = re.sub("\D", "", enginePower)
        except Exception: features['enginePower'] = np.NaN
    
        # equipment_dict
        equipment_dict = dict()
        try:
            complectation = ticket_bs.find('section',class_='CardComplectation')
            complectation_items = complectation.find_all('div',class_="ComplectationGroups__group")
            for item in complectation_items:
                item_name = item.find('span',class_="ComplectationGroups__itemName").text
                item_content_tag = item.find_all('li',class_="ComplectationGroups__itemContentEl")
                item_content = [x.text for x in item_content_tag]
                equipment_dict[item_name] = item_content 
            features['equipment_dict'] = equipment_dict
        except Exception: features['equipment_dict'] = np.NaN
    
        # fuelType
        try: features['fuelType'] = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[2]
        except Exception: features['fuel_type'] = np.NaN
        
        # mileage
        try:
            mileage = ticket_bs.find('li',class_='CardInfoRow_kmAge').find_all('span')[1].text
            features['mileage'] = re.sub("\D", "", mileage)
        except Exception: features['mileage'] = np.NaN
    
        # modelDate
        ## заполняется названием марки, модели и поколением модели - следует далее заменить на год модели/поколения
        try:
            modelDate_tag = ticket_bs.find_all('a',class_='CardBreadcrumbs__itemText')
            features['modelDate'] = (modelDate_tag[0].text.strip() + ' ' +
                                     modelDate_tag[1].text.strip() + ' ' +
                                     modelDate_tag[2].text.strip())  
        except Exception: features['modelDate'] = np.NaN
    
        # model_name
        try: features['model_name'] = ticket_bs.find_all('a',class_='CardBreadcrumbs__itemText')[1].text.strip()
        except Exception: features['model_name'] = np.NaN
     
        # numberOfDoors
        try:
            numberOfDoors_tag = ticket_bs.find('li',class_='CardInfoRow_bodytype').find('a')
            numberOfDoors_pre = re.findall('\d', numberOfDoors_tag.text)
            features['numberOfDoors'] = int(numberOfDoors_pre[0])
        except Exception: features['numberOfDoors'] = np.NaN
    
        # productionDate
        try: features['productionDate'] = ticket_bs.find('li',class_='CardInfoRow_year').find('a').text
        except Exception: features['productionDate'] = np.NaN
    
        # sell_id
        try:         
            invers_ticket_url = ticket_url[::-1]
            id_start, id_end = invers_ticket_url.find('/',1) , invers_ticket_url.find('-')+1
            features['sell_id'] = ticket_url[-id_start:-id_end]
        except Exception: features['sell_id'] = np.NaN
    
        # vehicleTransmission
        try:
            features['vehicleTransmission'] = (ticket_bs.find('li',class_='CardInfoRow_transmission').
                                               find_all('span')[1].text)
        except Exception: features['vehicleTransmission'] = np.NaN
            
        # vendor
        european = ['SKODA', 'AUDI',  'VOLVO', 'BMW', 'MERCEDES', 'VOLKSWAGEN']
        japanese = ['HONDA','NISSAN','TOYOTA','INFINITI',  'LEXUS', 'MITSUBISHI']
        if features['brand'] in european :  features['vendor'] = 'EUROPEAN'
        elif features['brand'] in japanese :  features['vendor'] = 'JAPANESE'
        else: features['vendor'] = 'NAN'
        
        # Владельцы
        try: features['Владельцы'] = ticket_bs.find('li',class_='CardInfoRow_ownersCount').find_all('span')[1].text
        except Exception: features['Владельцы'] = np.NaN

        # ПТС
        try: features['ПТС'] = ticket_bs.find('li',class_='CardInfoRow_pts').find_all('span')[1].text
        except Exception: features['ПТС'] = np.NaN
        
        # Привод
        try: features['Привод'] = ticket_bs.find('li',class_='CardInfoRow_drive').find_all('span')[1].text
        except Exception: features['Привод'] = np.NaN
    
        # Руль
        try: features['Руль'] = ticket_bs.find('li',class_='CardInfoRow_wheel').find_all('span')[1].text 
        except Exception: features['Руль'] = np.NaN
    
        # Привод
        try: features['Привод'] = ticket_bs.find('li',class_='CardInfoRow_drive').find_all('span')[1].text
        except Exception: features['Привод'] = np.NaN

        # Цена предложения
        try:
            offerprice = ticket_bs.find('span',class_='OfferPriceCaption__price').text
            features['offerprice'] = re.sub("\D", "", offerprice)
        except Exception: features['offerprice'] = np.NaN
            
    return features

## Установка параметров, определение констант

In [3]:
url_for_marks_models = 'https://auto.ru/catalog/cars/all/'

data_columns = ['bodytype', 'brand', 'car_url', 'color', 'engineDisplacement',
       'enginePower', 'equipment_dict', 'fuel_type', 'mileage', 'modelDate',
       'model_name', 'numberOfDoors', 'productionDate', 'sell_id', 'vehicleTransmission',
       'vendor', 'Владельцы', 'ПТС', 'Привод', 'Руль', 'offerprice']

headers = {
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
'Cache-Control': 'max-age=0',
'Connection': 'keep-alive',
'Cookie': 'suid=bf4a59ff8840503c2077bf716a7bbeef.295e234731ada0bc538a541689345348; tmr_lvid=d2115cf4bc0ee3e6bc3ad89db8fdac9d; tmr_lvidTS=1596630011564; _ym_uid=1596630015684249973; _ga=GA1.2.1113974574.1596630016; autoruuid=g5f949c622cu74bpb1qps6a12l6p5u7t.07262c19ce2a1f07cac53c4ce06ef69b; gids=213; gradius=200; mindboxDeviceUUID=a231f610-2f9e-4911-9cb8-9112afa13ab1; directCrm-session=%7B%22deviceGuid%22%3A%22a231f610-2f9e-4911-9cb8-9112afa13ab1%22%7D; tmr_reqNum=15; yuidlt=1; yandexuid=1471619051363959249; my=YysBgNU2AQEA; crookie=uCcId3jGlYfIvH+2UzzszOwTawIMGSzWumxanVb1Ras+Mu6qi+8yzj8EL+czftU9orxvkiKDKe/wgGsJDdJMlqyn6WU=; cmtchd=MTYxMjgyMjU3MTQ4MA==; _csrf_token=04c10009d8f17a08b4d507f70f2a287c28bf3c17f7b3ef42; gdpr=0; _ym_isad=2; index-selector-tab=marks; listing_view_session={}; listing_view=%7B%22output_type%22%3Anull%2C%22version%22%3A1%7D; autoru-visits-count=2; salon_phone_utms=utm_medium%3Dcpm%26utm_source%3Dauto-ru%26utm_campaign%3Dauto-ru_rus-r225_proauto-rk2021%26utm_content%3D113pa-100PRx40-otchety-o-proshlom-mashiny-ot-99-rublei_proauto-promo-page_rus-r225; hide-proauto-pimple=1; from=direct; autoru_sid=a%3Ag5f949c622cu74bpb1qps6a12l6p5u7t.07262c19ce2a1f07cac53c4ce06ef69b%7C1613427368824.604800.BYczsuAhOkO7E_tI9WN3ZQ.vyyvypdKR_sNAGlonPspJxUInUX2GwuD7owqbI5Sw58; X-Vertis-DC=vla; _ym_d=1613250934; from_lifetime=1613250934966; cycada=FXXHRKJxPTj6XyBIJ1I0Or150N9cGJGcL1yh7v8BaXc=',
'Host': 'auto.ru',
'sec-ch-ua': '"Chromium";v="88", "Google Chrome";v="88", ";Not A Brand";v="99"',
'sec-ch-ua-mobil': '?0',
'Sec-Fetch-Dest': 'document',
'Sec-Fetch-Mode': 'navigate',
'Sec-Fetch-Site': 'same-origin',
'Sec-Fetch-User': '?1',
'Upgrade-Insecure-Requests': '1',
'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
};


# Парсинг
## Парсинг вспомогательных данных
### Создаем и наполняем ***`marks_models `*** , получаем ***`marks_models_for_parsing`*** и ***`extra_marks_models`***
***marks_models*** - pd.Series в которм   
индекс:  обозначения марки на сайте auto.ru  
значения: списки всех моделей для каждой марки.

***marks_models_for_parsing*** - содержит только марки, присутсвующие в валидационном наборе данных **test**

***extra_marks_models*** - содержит все марки, за исключением присутсвующиx в валидационном наборе данных **test**

In [4]:
marks_models = get_marks_models()

In [5]:
marks_for_parsing = ['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI',
       'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI']
marks_models_for_parsing = marks_models.loc[marks_for_parsing]

extra_marks = marks_models.index.drop(marks_for_parsing)
extra_marks_models = marks_models.loc[extra_marks]

### Создаем и заполняем `model_generation_year`
***model_generation_year*** - pd.DataFrame  в котором:  
        *full_name* - полное название марки и модели с указанием поколения;   
        *bodytype* - тип кузова поколения модели;  
        *generation_year* - год начала выпуска поколения  


In [9]:
%%time
# model_generation_year_all =  get_model_generation_year(marks_models)
# model_generation_year_all.to_csv('./Project_5_data/model_generation_year_all.csv',index=False)
model_generation_year_all = pd.read_csv('./Project_5_data/model_generation_year_all.csv')

Wall time: 7.01 ms


In [58]:
# в полученном DataFrame встречаются дупликаты (иногда они отличаются годом начала выпуска поколения),
# количество их невелико,удалим повторы
model_generation_year_all = model_generation_year_all[
    ~model_generation_year_all.duplicated(subset =['full_name','bodytype'])]

# Парсинг набора данных для обучения
## Получение данных по маркам, присутсвующим в валидационном наборе данных 

In [7]:
'''
train = pd.DataFrame(columns=data_columns)

for mark in marks_models_for_parsing.index:
    for model in marks_models_for_parsing[mark]:
        model_url = 'https://auto.ru/moskva/cars/' + mark + '/' + model + '/used/'

    # просмотр последовательно всех страниц текущей модели-model текущей марки-mark
        for pages_num in range(1,100): 
        
            if pages_num==1: params = {}
            else:            params = {'page': pages_num}
    
            res = requests.get(model_url + '?output_type=table', params=params, headers=headers)
            res.encoding = 'utf-8'
            time.sleep(0.5)
    
            # создаем обьект bs4.BeautifulSoup из html очередной страницы с обьявлениями по текущей модели текущей марки
            soup = BeautifulSoup(res.text, 'html.parser')  
    
            # список html-ек карточек на странице                                               
            tickets_on_page_list = soup.find_all('a', class_='ListingItemTitle-module__link') 
    
            if not tickets_on_page_list: # выход по исчерпанию страниц текущей модели текущей марки
                break
        
            # обработка карточек на странице
            for ticket in tickets_on_page_list:
                # получаем url карточки текущего обьявления
                ticket_url = ticket.get('href')
    
                # извлекаем признаки и заполняем строку train-а
                train.loc[len(train)] = get_features_from_ticket(ticket_url)
                ''';

In [11]:
# train.to_csv('./Project_5_data/train',index=False)
train = pd.read_csv('./Project_5_data/train.csv')

## Получение данных по всем остальным ( т.е. за исключением присутсвующиx в валидационном наборе ) маркам

In [16]:
'''
extra_train = pd.DataFrame(columns=data_columns)

for mark in extra_marks_models.loc['KIA':].index:
    print(mark, len(extra_train), end='\n\t')
    for model in extra_marks_models.loc['KIA':][mark]:
        model_url = 'https://auto.ru/moskva/cars/' + mark + '/' + model + '/used/'

    # просмотр последовательно всех страниц текущей модели-model текущей марки-mark
        for pages_num in range(1,100): 
        
            if pages_num==1: params = {}
            else:            params = {'page': pages_num}
    
            res = requests.get(model_url + '?output_type=table', params=params, headers=headers)
            res.encoding = 'utf-8'
            time.sleep(0.5)
    
            # создаем обьект bs4.BeautifulSoup из html очередной страницы с обьявлениями по текущей модели текущей марки
            soup = BeautifulSoup(res.text, 'html.parser')  
    
            # список html-ек карточек на странице                                               
            tickets_on_page_list = soup.find_all('a', class_='ListingItemTitle-module__link') 
    
            if not tickets_on_page_list: # выход по исчерпанию страниц текущей модели текущей марки
                break
        
            # обработка карточек на странице
            for ticket in tickets_on_page_list:
                # получаем url карточки текущего обьявления
                ticket_url = ticket.get('href')
    
                # извлекаем признаки и заполняем строку train-а
                extra_train.loc[len(extra_train)] = get_features_from_ticket(ticket_url)
''';

In [12]:
# extra_train.to_csv('./Project_5_data/extra_train.csv',index=False)
extra_train = pd.read_csv('./Project_5_data/extra_train.csv')

In [11]:
# all_marks = train.append(extra_train, ignore_index=True)

In [32]:
# all_marks.to_csv('./Project_5_data/all_marks.csv',index=False)
all_marks = pd.read_csv('./Project_5_data/all_marks.csv')

# DEVELOPMENT

In [57]:
# model_url = 'https://auto.ru/moskva/cars/skoda/fabia/used/'

In [85]:
all_marks.head()

,bodyType,brand,car_url,color,engineDisplacement,enginePower,equipment_dict,fuelType,mileage,modelDate,...,numberOfDoors,productionDate,sell_id,vehicleTransmission,vendor,Владельцы,ПТС,Привод,Руль,offerprice
0,седан,SKODA,https://auto.ru/cars/used/sale/skoda/120/10858...,бежевый,1.2,53.0,{'Прочее': ['Полноразмерное запасное колесо']},Бензин,80000.0,"Skoda 105, 120 I",...,NaN,1984.0,1085892952,механическая,EUROPEAN,2 владельца,Оригинал,задний,Левый,170000.0
1,универсал 3 дв.,SKODA,https://auto.ru/cars/used/sale/skoda/1200/1102...,голубой,1.2,45.0,NaN,Бензин,18000.0,Skoda 1200 I,...,3.0,1960.0,1102072444,механическая,EUROPEAN,1 владелец,Оригинал,задний,Левый,400000.0
2,хэтчбек 5 дв.,SKODA,https://auto.ru/cars/used/sale/skoda/fabia/110...,пурпурный,1.6,105.0,{'Безопасность': ['Подушка безопасности водите...,Бензин,168000.0,Skoda Fabia II,...,5.0,2009.0,1102546031,автоматическая,EUROPEAN,2 владельца,Оригинал,передний,Левый,439000.0
3,хэтчбек 5 дв.,SKODA,https://auto.ru/cars/used/sale/skoda/fabia/110...,белый,1.6,105.0,{'Безопасность': ['Антипробуксовочная система ...,Бензин,66165.0,Skoda Fabia II Рестайлинг,...,5.0,2011.0,1102447624,автоматическая,EUROPEAN,1 владелец,Оригинал,передний,Левый,470000.0
4,хэтчбек 5 дв.,SKODA,https://auto.ru/cars/used/sale/skoda/fabia/110...,жёлтый,1.6,105.0,{'Безопасность': ['Антипробуксовочная система ...,Бензин,45748.0,Skoda Fabia II Рестайлинг,...,5.0,2013.0,1101532994,автоматическая,EUROPEAN,3 или более,Оригинал,передний,Левый,487767.0


In [121]:
        # Привод
#         try: features['Привод'] = ticket_bs.find('li',class_='CardInfoRow_drive').find_all('span')[1].text
#         except Exception: features['Привод'] = np.NaN/

def modelDate_convert(row):
    '''
    описание
    '''
    try: model_Date = model_generation_year_all.loc[(model_generation_year_all.full_name == row.modelDate) & (model_generation_year_all.bodytype == row.bodyType)].generation_year.iloc[0] 
    except Exception: model_Date = np.NaN
    return model_Date

In [112]:
all_marks.loc[63]

bodyType                                                             NaN
brand                                                                NaN
car_url                https://auto.ru/cars/used/sale/skoda/fabia/110...
color                                                                NaN
engineDisplacement                                                   NaN
enginePower                                                          NaN
equipment_dict                                                       NaN
fuelType                                                             NaN
mileage                                                              NaN
modelDate                                                            NaN
model_name                                                           NaN
numberOfDoors                                                        NaN
productionDate                                                       NaN
sell_id                                            

In [107]:
tmp = all_marks.head(100)
tmp.apply(modelDate_convert,axis = 1)

IndexError: ('single positional indexer is out-of-bounds', 'occurred at index 63')

In [89]:
row = all_marks.loc[149]
modelDate_convert(row)

bodyType                                                   хэтчбек 5 дв.
brand                                                              SKODA
car_url                https://auto.ru/cars/used/sale/skoda/fabia/110...
color                                                             чёрный
engineDisplacement                                                   1.2
enginePower                                                           60
equipment_dict                                                       NaN
fuelType                                                          Бензин
mileage                                                           206000
modelDate                                      Skoda Fabia II Рестайлинг
model_name                                                         Fabia
numberOfDoors                                                          5
productionDate                                                      2010
sell_id                                            